In [1]:
import requests, bs4
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
driver_loc = "./chromedriver"

In [2]:
from selenium.webdriver.common.action_chains import ActionChains
import time
sleepTime = 1
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('windows-size = 1920x1080')             # 적용은 안 되지만 headless때 혹시라도 도움될까 싶어서...
# options.add_argument('headless')                           # 리눅스처럼 웹 화면 없는 경우에도 실행 가능.
# options.add_argument('disable-gpu')
# options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")

driver = webdriver.Chrome(driver_loc, options=options)

In [4]:
# 1. 홈페이지로 이동
danawaUrl = 'http://www.enuri.com'
driver.get(danawaUrl)

# 2. 마우스 오버로 선택이 안 되고 클릭을 해야하기 때문에 분리. 메인 메뉴 클릭.
mainMenu = '//*[@id="gnbMenu"]/li[4]/a/em'
driver.find_element_by_xpath(mainMenu).click()
time.sleep(sleepTime)

# 3. 스마트워치 > 스마트워치 > 전체상품 보기 클릭
subMenu = '//*[@id="first_depth3"]/li[7]'
targetMenu = '//*[@id="first_depth3"]/li[7]/div/ul[2]/li[1]'

sub = driver.find_element_by_xpath(subMenu)
target = driver.find_element_by_xpath(targetMenu)

ActionChains(driver).move_to_element(sub).click(target).perform()
time.sleep(sleepTime)

# 가격비교 이동
compareXpath = '//*[@id="tabsDiv"]/li[2]/a'
compareMenu = driver.find_element_by_xpath(compareXpath)
compareMenu.click()
time.sleep(sleepTime)

# 90개씩 보기
viewXpath = '//*[@id="pageGapSelDiv"]/a'
viewMenu = driver.find_element_by_xpath(viewXpath)
viewMenu.click()
time.sleep(sleepTime)

ninetyXpath = '//*[@id="pageGapSelDiv"]/ul/li[3]/a'
ninetyMenu = driver.find_element_by_xpath(ninetyXpath)
ninetyMenu.click()
time.sleep(sleepTime)

# 4. 현재 페이지 소스 가져오기
html = driver.page_source.encode('utf-8')

bs = bs4.BeautifulSoup(html, 'html.parser')
# bs

In [5]:
# 상품명 구하기
name = bs.select('#listBodyDiv > li > div.sp_title > strong')
# name    # 디버깅용.

In [6]:
preFix = '위 '
nameList = []

for product in name:
    try:
        nameTmp = product.text.split('위 ')[1]    # re.split('위 ',name[0].text)[1] 로 바꿀 수도 있음.
        nameList.append(nameTmp)
    except Exception as e:
        nameTmp = product.text
        nameList.append(nameTmp)
    
# nameList    # 디버깅용.

In [7]:
# summary1 (무선랜, 화면크기, 블루투스, 내장OS)
summary1 = bs.select('#listBodyDiv > li > div.info_area > div.info_detail > div:nth-child(2)')
# summary1

In [8]:
summary1[0].text

'스마트워치|밴드교체형|무선랜(WiFi)|화면크기:1.4"(해상도:360x360,슈퍼AM-OLED)|터치스크린|블루투스5.0|내장OS:타이젠|호환가능OS:안드로이드5.0,iOS9.0이상\t'

# 액정 정보나 내장 OS 정보는 아래에는 나오지 않아서 위에서 긁어올까 싶어서 작성하다 중간에 멈췄는데요... 근데 정보 누락이 많아서 이걸 여기서 긁을지 다나와에서 긁어서 병합 하는걸 시도해볼지 생각해봅시다.

In [91]:
specList = []

if (summary1[0].text.find('무선랜') != -1):
    wifi = 1

In [91]:
if (summary1[0].text.find('화면크기') != -1):
    screen = 1

In [91]:
summary1[0].text.find('블루투스') != -1

In [91]:
summary1[0].text.find('내장OS') != -1

In [ ]:
wifi = []
screen = []
bluetooth = []
os = []



In [ ]:
#modelno_39729557 > div.info_area > div.info_detail > div:nth-child(3)

In [9]:
# summary2
summary2 = bs.select('#listBodyDiv > li > div.info_area > div.info_detail > div:nth-child(3)')
# summary2    # 디버깅용.

In [10]:
summary2[0].text

'[하드웨어]|램:768M|4G내장|[수신알림+내용확인]|SNS,문자,전화|[건강관리]|스트레스모니터링|이동거리측정|달리기|사이클링|수영모니터링|심박측정|칼로리측정|운동정보기록|수면패턴분석|만보계|[부가기능]|다중언어|GPS|원격촬영|음성명령|화면테마설정|자이로센서|NFC|무선충전|생활방수(IP68)|[사용시간]|사용시간:평균 60시간,최대 131시간|[기타규격]|배터리일체형(330mAh)|재질:스테인리스스틸|무게:42g\t'

# 위에 이름은 90개 정확히 잘 긁어와지는데 여기는 빈 곳이 있어요. 이걸 비어있는데가 어디인지 매칭을 시켜줘서 none 처리 해서 90개 리스트 인덱스 맞춰서 담아야 할 것 같은데 try except를 어떻게 써야할지 고민해봐야 할 것 같아요. 담은 다음에 이 문자열을 잘 분리해서 '하드웨어', '수신알림+내용확인','건강관리' 이런거 분류하면 되지 않을까 싶어요.

In [13]:
summary2List = []

for product in summary2:
    try:
        specsTmp = product.text
        summary2List.append(specsTmp)

SyntaxError: unexpected EOF while parsing (<ipython-input-13-fa697d0a4bfc>, line 6)

In [32]:
len(summary2)

87

In [31]:
summary2[30]

<div class="summary"><b>[하드웨어]</b><em>|</em><a class="dicLink att_txt_165799_0" href="JavaScript:">CPU</a>:1.15GHz,듀얼코어<em>|</em><a class="dicLink att_txt_165506_0" href="JavaScript:">램</a>:768M<em>|</em><a class="dicLink att_txt_165499_2" href="JavaScript:">4G내장</a><em>|</em><br/><b>[건강관리]</b><em>|</em>스트레스모니터링<em>|</em><a class="dicLink att_txt_193512_36" href="JavaScript:">호흡가이드</a><em>|</em>달리기<em>|</em>수영모니터링<em>|</em><a class="dicLink att_txt_193512_1" href="JavaScript:">심박측정</a><em>|</em><a class="dicLink att_txt_193512_2" href="JavaScript:">운동정보기록</a><em>|</em><a class="dicLink att_txt_193512_6" href="JavaScript:">수면패턴분석</a><em>|</em><br/><b>[부가기능]</b><em>|</em><a class="dicLink att_txt_198508_105" href="JavaScript:">심박센서</a><em>|</em>GPS<em>|</em>날씨<em>|</em>일정관리<em>|</em><a class="dicLink att_txt_198508_52" href="JavaScript:">자이로센서</a><em>|</em><a class="dicLink att_txt_198508_56" href="JavaScript:">NFC</a><em>|</em>고도센서<em>|</em>기압센서<em>|</em>속도계<em>|</em>스마트 캐디앱<em>|</em>그린

In [3]:
!pip install selenium